In [9]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import glob, sys
import pickle

[sys.path.append(f) for f in glob.glob('../utils/*')]
from preprocess import *
from plotting_utils import *
from lineplots import plotsd
from stabilizer_utils import *
from recalibration_utils import *
from session_utils import *

from CosineTuning import *

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import FactorAnalysis

min_nblocks    = 2
participant    = 'T5'


FILE_DIR       = '/oak/stanford/groups/henderj/ghwilson/nonstationarities/' + participant + '/'
fig_path       = '/home/users/ghwilson/projects/nonstationarities/figures/'
filelist       = glob.glob(FILE_DIR + 'historical/*')
filelist.extend(glob.glob(FILE_DIR + 'new/*'))

block_constraints = getBlockConstraints(FILE_DIR)

files          = get_Sessions(filelist, min_nblocks,  block_constraints = block_constraints)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from joblib import Parallel, delayed
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import warnings


def rotateVelocity(intention_vectors, decoder_velocity):
    '''Rotate velocity signals toward target, as in KF ReFIT. Can 
    view as rescaling the intention signal estimates based on ground-truth
    speed as well. Inputs are:
    
        intention_vectors (2D float) - time x 2 of point-to-target signals
        decoded_velocity (2D float)  - time x 2 of ground-truth cursor velocities
        '''
    
    # standardize intention estimates
    intention_vectors /= np.linalg.norm(intention_vectors, axis=1,keepdims=True) 
    
    # now rescale according to decoded speeds
    intention_vectors *= np.linalg.norm(decoder_velocity, axis=1, keepdims=True)
    
    return intention_vectors


def processSingleSession(file, cfg):
    
    n_restarts   = 10
    session_dict = dict()
    fields       = ['TX', 'cursorPos', 'targetPos', 'decVel']
    
    # First calculate encoding and decoding weights independently for each session:
    session = DataStruct(file, causal_filter = cfg['sigma'], alignScreens = True)
    blocks  = block_constraints[file]

    # obtain features and cursorError targets:
    session_dat = getTrainTest(session, fields = fields, train_size = cfg['train_size'], 
                               task = cfg['task'], blocks = blocks, returnFlattened = True) 
    session_dict['decoder_means'] = session_dat['test_TX'][-1].mean(axis = 0)
    
    train_vel = np.concatenate(session_dat['train_decVel'])
    train_y   = np.concatenate(session_dat['train_targetPos']) - np.concatenate(session_dat['train_cursorPos'])
    test_y    = np.concatenate(session_dat['test_targetPos']) - np.concatenate(session_dat['test_cursorPos'])
    train_x, test_x = subtractMeans(session_dat['train_TX'], session_dat['test_TX'], method = 'blockwise', concatenate = True)

    scores = np.zeros((2,))
    
    rotated_y = rotateVelocity(train_y, train_vel)
    
    scores[i]   = LinearRegression().fit(train_x, train_y).score(test_x, test_y)
    scores[i+1] = LinearRegression().fit(train_x, rotated_y).score(test_x, test_y)
    
    return scores

In [18]:
len(session_dat['train_decVel'][3])

IndexError: list index out of range

In [53]:
i = 49

n_restarts   = 10
session_dict = dict()
fields       = ['TX', 'cursorPos', 'targetPos', 'decVel']

# First calculate encoding and decoding weights independently for each session:
session = DataStruct(files[i], causal_filter = 2, alignScreens = True)
blocks  = block_constraints[files[i]]

# obtain features and cursorError targets:
session_dat = getTrainTest(session, fields = fields, train_size = cfg['train_size'], 
                           task = cfg['task'], blocks = blocks, returnFlattened = True) 
session_dict['decoder_means'] = session_dat['test_TX'][-1].mean(axis = 0)

train_vel = np.concatenate(session_dat['train_decVel'])
train_y   = np.concatenate(session_dat['train_targetPos']) - np.concatenate(session_dat['train_cursorPos'])
test_y    = np.concatenate(session_dat['test_targetPos']) - np.concatenate(session_dat['test_cursorPos'])
train_x, test_x = subtractMeans(session_dat['train_TX'], session_dat['test_TX'], method = 'blockwise', concatenate = True)

scores = np.zeros((2,))


rotated_y = rotateVelocity(np.copy(train_y), train_vel)
bad_idxs  = np.where(np.isnan(rotated_y))[0]
train_x   = np.delete(train_x, bad_idxs, axis=0)
rotated_y = np.delete(rotated_y, bad_idxs, axis=0)
train_y   = np.delete(train_y, bad_idxs, axis=0)


preds     = LinearRegression().fit(train_x, train_y).predict(test_x)
rot_preds = LinearRegression().fit(train_x, rotated_y).predict(test_x)

print(np.corrcoef(preds.flatten(), test_y.flatten())[0,1])
print(np.corrcoef(rot_preds.flatten(), test_y.flatten())[0,1])


/tmp/ipykernel_2577/1054599154.py:17: RuntimeWarning: invalid value encountered in divide
  intention_vectors /= np.linalg.norm(intention_vectors, axis=1,keepdims=True)


0.4737629572341507
0.42326213627049886


In [7]:
# general settings:
np.random.seed(42)
cfg = dict()

cfg['task']         = None
cfg['train_size']   = 0.5
cfg['sigma']        = 2

# --------------------------------------------------
#dict_list = list()
#for file in files:
#    dict_list = processSingleSession(file, cfg) 
#    print(file)

dict_list    = Parallel(n_jobs=10, verbose = 11)(delayed(processSingleSession)(file, cfg) for file in files)
session_dict = dict()

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:    7.5s
[Parallel(n_jobs=10)]: Done   2 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done   3 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done   4 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done   6 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done   7 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done   9 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done  10 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done  11 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done  13 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done  14 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done  15 tasks      | elapsed:  

ValueError: operands could not be broadcast together with shapes (22249,2) (488,1) (22249,2) 